In [ ]:
from google.colab import drive
drive.mount('/content/drive')
#/content/drive/MyDrive/

Mounted at /content/drive


In [ ]:
!pip install neo4j neo4j-graphrag -q
!pip install langchain-community onnxruntime -q
!pip install sentence-transformers -q
!pip install rouge_score -q

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 312.3/312.3 kB 7.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 180.4/180.4 kB 11.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 183.9/183.9 kB 8.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 304.2/304.2 kB 19.5 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2025.3.2 requires fsspec==2025.3.2, but you have fsspec 2024.12.0 which is incompatible.
torch 2.6.0+cu124 requires nvidia-cublas-cu12==12.4.5.8; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cublas-cu12 12.5.3.2 which is incompatible.
torch 2.6.0+cu124 requires nvidia-cuda-cupti-cu12==12.4.127; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cuda-cupti-cu12 12.5.82 which is incompatible.
torch 2.6.0+cu124 requires nvidia-cud

In [ ]:
import re
import ast
import pandas as pd
import random
from neo4j import GraphDatabase
from neo4j import GraphDatabase
from collections import defaultdict
from collections import defaultdict, Counter
from neo4j_graphrag.generation import GraphRAG
from neo4j_graphrag.llm import OpenAILLM
from neo4j_graphrag.retrievers import VectorRetriever
from typing import List
from neo4j.exceptions import Neo4jError
from tqdm import tqdm
from sentence_transformers import SentenceTransformer
from langchain.schema import Document
from langchain.text_splitter import RecursiveCharacterTextSplitter

## LOAD

In [ ]:
driver_benh = GraphDatabase.driver("neo4j+s://4881e1c6.databases.neo4j.io", auth=("neo4j", ""))
driver_thuoc = GraphDatabase.driver("neo4j+s://fb4f9b28.databases.neo4j.io", auth=("neo4j", ""))

In [ ]:
model_embed = SentenceTransformer('keepitreal/vietnamese-sbert')

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/229 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/122 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/3.90k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

vocab.txt:   0%|          | 0.00/895k [00:00<?, ?B/s]

bpe.codes:   0%|          | 0.00/1.14M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/17.0 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/150 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

## TRUY XUẤT TĂNG CƯỜNG TỪ ĐỒ THỊ

## Xác định keyword

In [ ]:
import re

multi_word_stopwords = ['dấu hiệu', 'thế nào']
single_word_stopwords = set([
    'là', 'của', 'và', 'hoặc', 'có', 'trong',
    'các', 'những', 'đó', 'đây', 'ra', 'gì', 'theo',
    'để', 'cho', 'thuốc', 'như'
])

def remove_stopwords(phrase):
    # Loại bỏ dấu câu ? . ! ở cuối cụm
    phrase = re.sub(r'[?.!]+$', '', phrase).strip()

    phrase_lower = phrase.lower()

    # Loại bỏ multi-word stopwords
    for mw in multi_word_stopwords:
        pattern = r'\b' + re.escape(mw) + r'\b'
        phrase_lower = re.sub(pattern, '', phrase_lower)

    # Loại bỏ khoảng trắng thừa
    phrase_lower = re.sub(r'\s+', ' ', phrase_lower).strip()

    # Bỏ dấu câu và tách từ
    phrase_clean = re.sub(r'[^\w\s]', '', phrase_lower)
    tokens = phrase_clean.split()

    # Lọc từ đơn
    filtered_tokens = [t for t in tokens if t not in single_word_stopwords]

    if not filtered_tokens:
        return ''

    # Khôi phục từ gốc từ câu gốc
    original_words = phrase.split()
    filtered_original = []
    for w in original_words:
        w_clean = re.sub(r'[^\w\s]', '', w.lower())
        if w_clean in filtered_tokens:
            filtered_original.append(w)
            filtered_tokens.remove(w_clean)

    return ' '.join(filtered_original)


def sentence_to_keywords(text):
    results = []
    seen = set()

    # Tách thành các cụm theo dấu phẩy, nhưng không tách trong ngoặc
    phrases = re.split(r',(?![^(]*\))', text.strip())
    phrases = [p.strip() for p in phrases if p.strip()]
    filtered_phrases = []

    for phrase in phrases:
        cleaned = remove_stopwords(phrase)
        if cleaned and cleaned not in seen:
            filtered_phrases.append(cleaned)

    max_len = len(filtered_phrases)

    if max_len == 1:
        single = filtered_phrases[0]
        seen.add(single)
        results.append(single)

    elif max_len > 1:
        # Ghép từ 2 đến N cụm liên tiếp
        for n in range(2, max_len + 1):
            for i in range(max_len - n + 1):
                merged = ', '.join(filtered_phrases[i:i + n])
                if merged not in seen:
                    seen.add(merged)
                    results.append(merged)

    return results


In [ ]:
question = 'thế nào là bệnh Nhồi máu cơ tim?'
sentence_to_keywords(question)

['bệnh Nhồi máu cơ tim']

In [ ]:
def embed_nodes_and_create_vector_index(driver, labels, embedding_model, index_name, dim=768):
    """
    Sinh embedding lại toàn bộ node có label được chỉ định (trừ Chunk/Document)
    và chỉ xử lý node có embedding bị sai chiều hoặc chưa có embedding.
    """

    exclude_labels = {"Chunk", "Document"}

    with driver.session() as session:
        for label in labels:
            if label.lower() in exclude_labels:
                print(f"Skipping label '{label}'")
                continue

            # Lấy node có id, và hoặc embedding bị thiếu hoặc sai chiều (khác dim)
            result = session.run(f"""
                MATCH (n:`{label}`)
                WHERE n.id IS NOT NULL AND (
                    n.embedding IS NULL OR size(n.embedding) <> $dim
                )
                RETURN id(n) AS node_id, n.id AS text
            """, dim=dim)

            nodes = [(record["node_id"], record["text"]) for record in result]

            if not nodes:
                print(f"All nodes with label '{label}' have valid embeddings.")
                continue

            # Sinh embedding
            texts = [text for _, text in nodes]
            embeddings = embedding_model.encode(texts, normalize_embeddings=True).tolist()

            # Cập nhật embedding mới vào graph
            for (node_id, _), embedding in tqdm(zip(nodes, embeddings), total=len(nodes), desc=f"→ Updating {label}"):
                session.run("""
                    MATCH (n)
                    WHERE id(n) = $node_id
                    SET n.embedding = $embedding
                    SET n:`__Entity__`
                """, node_id=node_id, embedding=embedding)

        # Xoá index cũ nếu có
        session.run(f"DROP INDEX {index_name} IF EXISTS")

        # Tạo index mới
        session.run(f"""
            CALL db.index.vector.createNodeIndex(
                $index_name,
                '__Entity__',
                'embedding',
                $dim,
                'cosine'
            )
        """, index_name=index_name, dim=dim)
## embedding các node bệnh
labels_benh = ['Chủ đề', 'Tiêu đề', 'Nội dung', 'BỆNH', 'THUỘC_TÍNH_BỆNH', 'NỘI_DUNG_THUỘC_TÍNH_BỆNH']
embed_nodes_and_create_vector_index(driver_benh, labels_benh, model_embed, 'benh_embedding', dim=768)

## embedding các node thuốc
labels_thuoc = ['THUỐC', 'THUỘC_TÍNH_THUỐC', 'NỘI_DUNG_THUỘC_TÍNH_THUỐC']
embed_nodes_and_create_vector_index(driver_thuoc, labels_thuoc, model_embed, 'thuoc_embedding', dim=768)

All nodes with label 'Chủ đề' have valid embeddings.


All nodes with label 'Tiêu đề' have valid embeddings.


All nodes with label 'Nội dung' have valid embeddings.


All nodes with label 'BỆNH' have valid embeddings.


All nodes with label 'THUỘC_TÍNH_BỆNH' have valid embeddings.


All nodes with label 'NỘI_DUNG_THUỘC_TÍNH_BỆNH' have valid embeddings.


All nodes with label 'THUỐC' have valid embeddings.


All nodes with label 'THUỘC_TÍNH_THUỐC' have valid embeddings.


All nodes with label 'NỘI_DUNG_THUỘC_TÍNH_THUỐC' have valid embeddings.


## Tìm kiếm node_id theo keyword

In [ ]:
# Bọc mô hình thành đúng interface neo4j_graphrag yêu cầu
class CustomEmbedder:
    def __init__(self, model: SentenceTransformer):
        self.model = model

    def embed_query(self, text: str) -> List[float]:
        embedding = self.model.encode(text, convert_to_numpy=True)
        return embedding.tolist()

In [ ]:
def search_queries(entity_group, query_list, top_k=5):
    # Create an Embedder object
    embedder = CustomEmbedder(model_embed)

    # Initialize the retriever
    retriever_benh = VectorRetriever(driver_benh, 'benh_embedding', embedder)
    retriever_thuoc = VectorRetriever(driver_thuoc, 'thuoc_embedding', embedder)

    # Chọn retriever tương ứng
    retriever = retriever_thuoc if entity_group.upper() == 'THUỐC' else retriever_benh

    results_dict = {}
    all_results = []

    # Truy vấn từng từ khóa
    for query_text in query_list:
        results = retriever.search(query_text=query_text, top_k=top_k)
        results_dict[query_text] = results.items
        all_results.extend(results.items)

    # Tìm ngưỡng phù hợp
    threshold = 0.7  # mặc định
    if any(item.metadata.get("score", 0) > 0.9 for item in all_results):
        threshold = 0.9
    elif any(item.metadata.get("score", 0) > 0.8 for item in all_results):
        threshold = 0.8

    # Lọc kết quả theo ngưỡng đã chọn
    high_score_results = [
        item for item in all_results
        if item.metadata.get("score", 0) > threshold
    ]

    # print(f"\nKết quả cho truy vấn: {query_text}\n" + "-" * 50)
    # for item in results.items:
    #     print(f"ID: {item.metadata['id']}")
    #     print(f"Nội dung: {item.content}")
    #     print(f"Điểm số: {item.metadata['score']}")
    #     print(f"Nhãn nút: {item.metadata['nodeLabels']}")
    #     print("-" * 40)

    return results_dict, high_score_results


In [ ]:
def get_existing_labels(driver):
    """Trả về tập hợp tất cả label hiện có trong DB."""
    with driver.session() as session:
        result = session.run("CALL db.labels()")
        return {record["label"] for record in result}

def find_parent_node(session, node_id):
    """
    Tìm node cha của node nội dung.
    """
    query = """
    MATCH (parent)-[*1..2]->(child)
    WHERE child.id = $node_id
    RETURN parent.id AS parent_id, labels(parent) AS parent_labels
    LIMIT 1
    """
    result = session.run(query, node_id=node_id)
    record = result.single()
    if record:
        return record['parent_id'], record['parent_labels']
    return None, None

def find_entities_from_retriever_results(entity_group, high_score_results, target_field='id'):
    """
    Tìm các thực thể BỆNH và THUỐC từ kết quả Retriever.
    Trả về danh sách [(id, [list các node trùng])], đã sort theo độ dài list giảm dần.
    Nếu [list các node trùng] chứa các node là Nội dung, NỘI_DUNG_THUỘC_TÍNH_BỆNH, NỘI_DUNG_THUỘC_TÍNH_THUỐC thì tìm về node phía trước nó.
    """
    match_map = defaultdict(list)

    # Chọn driver tương ứng
    if entity_group.upper() == 'THUỐC':
        driver = driver_thuoc
    else:
        driver = driver_benh

    # Lấy các label thực tế có trong DB
    existing_labels = get_existing_labels(driver)

    # Chỉ giữ lại những entity_type có tồn tại trong DB
    candidate_entity_types = ['BỆNH', 'THUỐC', 'Chủ đề']
    valid_entity_types = [et for et in candidate_entity_types if et in existing_labels]

    with driver.session() as session:
        for item in high_score_results:
            try:
                content_data = ast.literal_eval(item.content)
            except (ValueError, SyntaxError):
                continue

            node_id = content_data.get('id')
            labels = item.metadata.get('nodeLabels', [])
            target_labels = [label for label in labels if label != '__Entity__']

            if not node_id or not target_labels:
                continue

            # Nếu là node nội dung, tìm node cha
            if any(label in ['Nội dung', 'NỘI_DUNG_THUỘC_TÍNH_BỆNH', 'NỘI_DUNG_THUỘC_TÍNH_THUỐC'] for label in target_labels):
                parent_id, parent_labels = find_parent_node(session, node_id)
                if parent_id and parent_labels:
                    node_id = parent_id
                    target_labels = [label for label in parent_labels if label != '__Entity__']

            for target_label in target_labels:
                for entity_type in valid_entity_types:
                    query = f"""
                    MATCH (e:`{entity_type}`)-[*1..2]->(n:`{target_label}`)
                    WHERE toLower(n.{target_field}) = toLower($value)
                    RETURN e.id AS id
                    UNION
                    MATCH (e:`{entity_type}`)
                    WHERE toLower(e.{target_field}) = toLower($value)
                    RETURN e.id AS id
                    """
                    result = session.run(query, value=node_id)
                    for record in result:
                        matched_id = record['id']
                        match_map[matched_id].append(node_id)  # lưu lại các node trùng với thực thể

    # Trả về danh sách (id, [list các node trùng]) được sort theo độ dài list giảm dần
    return sorted(match_map.items(), key=lambda x: len(x[1]), reverse=True)

In [ ]:
question = 'Triệu chứng, khái niệm và nguyên nhân của bệnh Sốt rét?'
# Gọi hàm tìm kiếm
results_dict, high_score_results = search_queries('BỆNH', sentence_to_keywords(question))
result_find = find_entities_from_retriever_results('BỆNH',high_score_results)
result_find

[('Sốt rét',
  ['Triệu chứng bệnh Sốt rét',
   'Khái niệm bệnh Sốt rét',
   'Nguyên nhân bệnh Sốt rét']),
 ('Sốt siêu vi',
  ['Nguyên nhân bệnh Sốt siêu vi', 'Khái niệm bệnh Sốt siêu vi'])]

## Agent

In [ ]:
# # MẤT PHÍ
# from langchain.chat_models import ChatOpenAI
# from langchain.schema import HumanMessage, SystemMessage
# import os

# # Cấu hình API key và endpoint của OpenRouter
# os.environ["OPENAI_API_KEY"] = "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"
# os.environ["OPENAI_BASE_URL"] = "https://openrouter.ai/api/v1"

# def filter_question(question, model_name="openai/gpt-4o-mini"):
#     llm = ChatOpenAI(
#         temperature=0,
#         model_name=model_name,
#         openai_api_base=os.environ["OPENAI_BASE_URL"],
#     )
#     response = llm([
#         SystemMessage(content="Xác định xem câu hỏi sau về THUỐC hay BỆNH. Chỉ cần trả lời THUỐC hoặc BỆNH"),
#         HumanMessage(content=question)
#     ])
#     return response.content.strip()


In [ ]:
# MIỄN PHÍ
from langchain.chat_models import ChatOpenAI
from langchain.schema import HumanMessage, SystemMessage
import os
os.environ["OPENAI_API_KEY"] = ""

def filter_question(question, model_name="gpt-4o-mini"):
    llm = ChatOpenAI(model_name=model_name, temperature=0)
    response = llm([
        SystemMessage(content="Xác định xem câu hỏi sau liên quan đến THUỐC hay BỆNH gì. Trả lời dạng: THUỐC hoặc BỆNH "), #[THUỐC hoặc BỆNH, Tên THUỐC hoặc BỆNH]
        HumanMessage(content=question)
    ])
    return response.content.strip()

## Lấy context từ GRAPH

In [ ]:
def get_label_of_entity(entity_id, driver):
    """
    Lấy danh sách nhãn (labels) của node có id tương ứng, bỏ qua '__ENTITY__'.
    Trả về danh sách labels (ví dụ: ['BỆNH'], ['CHỦ_ĐỀ'], ...).
    """

    with driver.session() as session:
        query = (
            "MATCH (n) "
            "WHERE n.id = $entity_id "
            "RETURN labels(n) AS labels "
            "LIMIT 1"
        )
        result = session.run(query, entity_id=entity_id)
        record = result.single()
        if record:
            labels = record["labels"]
            filtered_labels = [label for label in labels if label != "__Entity__"]
            return filtered_labels
    return []

def get_related_nodes(entity_id, entity_group):
    """
    Lấy danh sách các node có quan hệ outgoing với node có id là entity_id,
    tự động xác định label của node này và truy vấn theo label đó.
    """
    if entity_group.upper() == 'THUỐC':
        driver = driver_thuoc
    else:
        driver = driver_benh

    labels = get_label_of_entity(entity_id, driver)

    if not labels:
        return []
    label = labels[0]  # Chọn label đầu tiên (sau khi đã loại bỏ __Entity__)

    with driver.session() as session:
        query = (
            f"MATCH (n:`{label}` {{id: $entity_id}})-[r]->(m) "
            "RETURN DISTINCT m.id AS node_name"
        )
        result = session.run(query, entity_id=entity_id)
        related_nodes = [record["node_name"] for record in result]
        return related_nodes


In [ ]:
def create_context_from_top_results(top_results, entity_group="BỆNH"):
    def capitalize_sentences(text):
        sentences = re.split(r'(?<=[.!?])\s+', text.strip())
        return ' '.join(s.capitalize() for s in sentences)

    contexts = []

    for topic_name, related_nodes in top_results:
        def build_context(nodes):
            parts = []
            khainiem_text = None
            for attr in nodes:
                # Bỏ qua nếu node có label là BỆNH, THUỐC hoặc CHỦ_ĐỀ
                labels = get_label_of_entity(attr, driver_thuoc if entity_group == "THUỐC" else driver_benh)
                if not labels or any(lbl.upper() in {"BỆNH", "THUỐC", "Chủ đề"} for lbl in labels):
                    continue

                details = get_related_nodes(attr, entity_group)
                if details:
                    clean_details = [d.rstrip('.').strip().lower() for d in details]
                    attr_clean = attr.strip().lower()

                    if attr_clean.startswith("khái niệm"):
                        khainiem_text = ", ".join(sorted(set(clean_details)))
                    else:
                        # Ghép mỗi 2 chi tiết thành 1 câu
                        for i in range(0, len(clean_details), 2):
                            chunk = clean_details[i:i+2]
                            # Chuyển dấu chấm thành dấu phẩy trong từng detail trước khi merge
                            chunk = [d.replace('.', ',') for d in chunk]
                            merged = ", ".join(sorted(set(chunk)))
                            parts.append(f"{attr_clean} là {merged}")
            if khainiem_text:
                parts.insert(0, khainiem_text)
            context = ". ".join(parts) + "."
            return capitalize_sentences(context)

        # Tạo context chính
        main_context = build_context(related_nodes)

        # Đếm số câu trong main_context
        sentence_count = len([s for s in re.split(r'[.!?]\s*', main_context) if s.strip()])

        extra_sentence = ""
        if sentence_count < 3:
            all_nodes = get_related_nodes(topic_name, entity_group)
            extra_nodes = [n for n in all_nodes if n not in related_nodes]
            for attr in extra_nodes:
                labels = get_label_of_entity(attr, driver_thuoc if entity_group == "THUỐC" else driver_benh)
                if not labels or any(lbl.upper() in {"BỆNH", "THUỐC", "Chủ đề"} for lbl in labels):
                    continue

                details = get_related_nodes(attr, entity_group)
                if details:
                    clean_details = [d.strip().lower().replace('.', ',').rstrip(',').strip() for d in details]
                    merged_detail = ", ".join(sorted(set(clean_details)))
                    attr_clean = attr.strip().lower()
                    extra_sentence = capitalize_sentences(f"{attr_clean} là {merged_detail}.")
                    break

        # Gộp context đầy đủ
        full_context = main_context + " " + extra_sentence if extra_sentence else main_context
        contexts.append(full_context)

    return contexts


In [ ]:
def get_top_topic_contexts(question):
    results_dict, high_score_results = search_queries(
        'BỆNH',
        sentence_to_keywords(question)
    )
    # tìm kiếm topics với node có điểm cao nhất
    max_score = max((item.metadata.get('score', 0) for item in high_score_results), default=0)
    highest_score_results = [item for item in high_score_results if item.metadata.get('score', 0) == max_score]
    topic_highest_score = find_entities_from_retriever_results(
        'BỆNH',
        highest_score_results
    )

    # tìm kiếm topics có count nhiều nhất
    highest_score_results = find_entities_from_retriever_results(
        'BỆNH',
        high_score_results
    )
    max_count = max(len(matches) for _, matches in highest_score_results)
    topic_highest_count = [
        (entity_id, matches)
        for entity_id, matches in highest_score_results
        if len(matches) == max_count
    ]

    # Gộp 2 danh sách kết quả theo entity_id
    merged_result = defaultdict(set)
    for entity_id, matches in topic_highest_score + topic_highest_count:
        merged_result[entity_id].update(matches)
    result = [(entity_id, list(matches)) for entity_id, matches in merged_result.items()]

    # Tạo danh sách context
    contexts = create_context_from_top_results(result, entity_group="BỆNH")

    documents = [
        Document(page_content=context, metadata={"source": idx})
        for idx, context in enumerate(contexts)
    ]

    # Tách văn bản thành các chunk nhỏ
    text_splitter_dot = RecursiveCharacterTextSplitter(
        chunk_size=412,
        chunk_overlap=0,
        separators=["."]
    )

    text_splitter_comma = RecursiveCharacterTextSplitter(
        chunk_size=412,
        chunk_overlap=0,
        separators=[","]
    )

    # Bước 1: Chia theo dấu chấm "."
    dot_chunks = text_splitter_dot.split_documents(documents)

    # Bước 2: Tiếp tục chia từng đoạn theo dấu phẩy ","
    final_chunks = []
    for chunk in dot_chunks:
        smaller_chunks = text_splitter_comma.split_documents([chunk])
        final_chunks.extend(smaller_chunks)

    # Trả về nội dung văn bản đã chia nhỏ
    return [doc.page_content for doc in final_chunks]


In [ ]:
test_questions = 'viêm nang lông là gì?	'

In [ ]:
get_top_topic_contexts(test_questions)

## ĐÁNH GIÁ TRUY XUẤT CỦA GRAPH

In [ ]:
# Đọc dữ liệu gốc
df_vinmec_qa = pd.read_csv('/content/drive/MyDrive/ChatBot/data/vinmec_qa.csv')

# Lấy 1000 dòng ngẫu nhiên
df_sample = df_vinmec_qa.sample(n=1000, random_state=42).reset_index(drop=True)

# Đổi tên cột 'context' thành 'ground_truth_context'
df_sample.rename(columns={'context': 'ground_truth_context'}, inplace=True)

# Hàm xử lý context trả về từ get_top_topic_contexts
def extract_context_text(context):
    if isinstance(context, list):
        return " ".join(doc.page_content if hasattr(doc, "page_content") else str(doc) for doc in context)
    return str(context)

# Hàm an toàn để truy xuất context, nếu lỗi hoặc không có thì trả về "Không tìm thấy"
def safe_get_context(q):
    try:
        result = get_top_topic_contexts(q)
        if not result:  # Nếu kết quả rỗng
            return "Không tìm thấy"
        return extract_context_text(result)
    except Exception as e:
        print(f"❌ Error retrieving context for question: {q}\n--> {e}")
        return "Không tìm thấy"

# Áp dụng hàm truy xuất an toàn
df_sample['retrieved_context'] = df_sample['question'].apply(safe_get_context)

# Xem kết quả
df_sample[['question', 'answer', 'ground_truth_context', 'retrieved_context']].head()

In [ ]:
from rouge_score import rouge_scorer
from nltk.translate.bleu_score import sentence_bleu, SmoothingFunction

def evaluate_context_retrieval(retrieved_contexts, ground_truth_contexts):
    assert len(retrieved_contexts) == len(ground_truth_contexts), "Số lượng context không khớp"

    rouge = rouge_scorer.RougeScorer(['rouge1', 'rouge2', 'rougeL'], use_stemmer=True)
    smooth_fn = SmoothingFunction().method4

    rouge_scores = {'rouge1': 0, 'rouge2': 0, 'rougeL': 0}
    bleu_scores = []

    for retrieved, ground_truth in zip(retrieved_contexts, ground_truth_contexts):
        # Tính ROUGE
        scores = rouge.score(ground_truth, retrieved)
        rouge_scores['rouge1'] += scores['rouge1'].fmeasure
        rouge_scores['rouge2'] += scores['rouge2'].fmeasure
        rouge_scores['rougeL'] += scores['rougeL'].fmeasure

        # Tính BLEU
        reference = [ground_truth.split()]
        candidate = retrieved.split()
        bleu = sentence_bleu(reference, candidate, smoothing_function=smooth_fn)
        bleu_scores.append(bleu)

    num_examples = len(retrieved_contexts)
    avg_rouge_scores = {key: value / num_examples for key, value in rouge_scores.items()}
    avg_bleu_score = sum(bleu_scores) / num_examples

    return avg_rouge_scores, avg_bleu_score

In [ ]:
retrieved_contexts = df_sample['retrieved_context'].tolist()   # context do RAG truy xuất
ground_truth_contexts = df_sample['ground_truth_context'].tolist()  # context chuẩn từ dữ liệu

avg_rouge, avg_bleu = evaluate_context_retrieval(retrieved_contexts, ground_truth_contexts)

print("Average ROUGE:", avg_rouge)
print("Average BLEU:", avg_bleu)


## ĐÁNH GIÁ TRUY XUẤT CỦA RAG VỚI FAISS

In [ ]:
# contexts = df_vinmec_context['context'].unique()[:2]
contexts = df_vinmec_context['context'].unique()

# Tạo danh sách Document chỉ từ 2 context đó
documents = [Document(page_content=context, metadata={"source": index})
             for index, context in enumerate(contexts)]

# Tách văn bản
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
texts = text_splitter.split_documents(documents)

In [ ]:
# Define the CustomEmbedder class to wrap the SentenceTransformer
class CustomSentenceTransformerEmbeddings(Embeddings):
    def __init__(self, model: SentenceTransformer):
        self.model = model

    def embed_documents(self, texts: List[str]) -> List[List[float]]:
        return self.model.encode(texts, convert_to_numpy=True).tolist()

    def embed_query(self, text: str) -> List[float]:
        return self.model.encode(text, convert_to_numpy=True).tolist()

In [ ]:
embedding_model = SentenceTransformer('keepitreal/vietnamese-sbert')
custom_embedder = CustomSentenceTransformerEmbeddings(embedding_model)

# Tạo FAISS vectorstore
faiss_vector_store = FAISS.from_documents(texts, custom_embedder)
faiss_retriever = faiss_vector_store.as_retriever(search_kwargs={"k": 2})

In [ ]:
from tqdm.notebook import tqdm
tqdm.pandas()

# Hàm lấy văn bản từ danh sách Document trả về
def retrieve_context_from_faiss(query, retriever, k=2):
    docs = retriever.get_relevant_documents(query)
    return " ".join(doc.page_content for doc in docs[:k])  # Ghép nội dung của top-k docs

# Lấy mẫu 100 câu hỏi
df_sample = df_vinmec_qa.sample(n=1000, random_state=42).reset_index(drop=True)

# Đổi tên cột context để giữ làm ground-truth
df_sample.rename(columns={'context': 'ground_truth_context'}, inplace=True)

# Áp dụng FAISS retriever lên từng câu hỏi
df_sample['retrieved_context'] = df_sample['question'].progress_apply(
    lambda q: retrieve_context_from_faiss(q, faiss_retriever, k=2)
)

# Xem kết quả
df_sample[['question', 'answer', 'ground_truth_context', 'retrieved_context']].head()


In [ ]:
from rouge_score import rouge_scorer
from nltk.translate.bleu_score import sentence_bleu, SmoothingFunction

def evaluate_context_retrieval(retrieved_contexts, ground_truth_contexts):
    assert len(retrieved_contexts) == len(ground_truth_contexts), "Số lượng context không khớp"

    rouge = rouge_scorer.RougeScorer(['rouge1', 'rouge2', 'rougeL'], use_stemmer=True)
    smooth_fn = SmoothingFunction().method4

    rouge_scores = {'rouge1': 0, 'rouge2': 0, 'rougeL': 0}
    bleu_scores = []

    for retrieved, ground_truth in zip(retrieved_contexts, ground_truth_contexts):
        # Tính ROUGE
        scores = rouge.score(ground_truth, retrieved)
        rouge_scores['rouge1'] += scores['rouge1'].fmeasure
        rouge_scores['rouge2'] += scores['rouge2'].fmeasure
        rouge_scores['rougeL'] += scores['rougeL'].fmeasure

        # Tính BLEU
        reference = [ground_truth.split()]
        candidate = retrieved.split()
        bleu = sentence_bleu(reference, candidate, smoothing_function=smooth_fn)
        bleu_scores.append(bleu)

    num_examples = len(retrieved_contexts)
    avg_rouge_scores = {key: value / num_examples for key, value in rouge_scores.items()}
    avg_bleu_score = sum(bleu_scores) / num_examples

    return avg_rouge_scores, avg_bleu_score


In [ ]:
retrieved_contexts = df_sample['retrieved_context'].tolist()   # context do RAG truy xuất
ground_truth_contexts = df_sample['ground_truth_context'].tolist()  # context chuẩn từ dữ liệu

avg_rouge, avg_bleu = evaluate_context_retrieval(retrieved_contexts, ground_truth_contexts)

print("Average ROUGE:", avg_rouge)
print("Average BLEU:", avg_bleu)


## GraphRAG + BARTpho

In [ ]:
import torch
import numpy as np
import re
from transformers import PreTrainedTokenizerFast
import onnxruntime as ort

def load_session_and_tokenizer(onnx_model_path: str, tokenizer_path: str):
    """Load the fine-tuned BARTpho model and tokenizer."""
    tokenizer = PreTrainedTokenizerFast.from_pretrained(tokenizer_path, local_files_only=True)
    session = ort.InferenceSession(onnx_model_path)
    return session, tokenizer

def generate_answer(question: str, contexts: list, session, tokenizer) -> str:
    answers = []
    contexts_to_process = contexts[:2] if len(contexts) > 2 else contexts
    for context in contexts_to_process:
        context = context.replace('_', ' ')
        inputs = tokenizer(question, context, return_tensors="pt")
        input_ids = inputs["input_ids"]
        attention_mask = inputs["attention_mask"]

        onnx_inputs = {
            "input_ids": input_ids.cpu().numpy(),
            "attention_mask": attention_mask.cpu().numpy()
        }

        start_logits, end_logits = session.run(["start_logits", "end_logits"], onnx_inputs)
        start_idx = np.argmax(start_logits[0])
        end_idx = np.argmax(end_logits[0]) + 1
        answer_ids = input_ids[0][start_idx:end_idx]
        answer = tokenizer.decode(answer_ids, skip_special_tokens=True)
        answer = answer.replace("<s>", "").replace("</s>", "")
        answers.append(answer.strip())

    combined_answer = ". ".join(filter(None, answers))
    if not combined_answer:
        combined_answer = "Không tìm thấy câu trả lời phù hợp"
    return combined_answer

def remove_whitespace_around_dot(text):
    return re.sub(r'\s*\.\s*', '.', text)

def test_generate_answer(question: str, contexts: list, onnx_model_path: str, tokenizer_path: str):
    question = remove_whitespace_around_dot(question)
    session, tokenizer = load_session_and_tokenizer(onnx_model_path, tokenizer_path)
    answer = generate_answer(question, contexts, session, tokenizer)
    return answer


In [ ]:
def merge_similar_sentences(answer):
    # Tách câu bằng dấu chấm, dấu hỏi, dấu chấm than
    sentences = [s.strip() for s in re.split(r'[.?!]', answer) if s.strip()]
    grouped = defaultdict(list)
    for sentence in sentences:
        words = sentence.split()
        if "là" in words:
            idx = words.index("là") + 1  # lấy cả từ "là"
            prefix = ' '.join(words[:idx])
            suffix = ' '.join(words[idx:]) if idx < len(words) else ''
            grouped[prefix].append(suffix.strip())
        else:
            # Nếu không có "là", dùng cả câu làm prefix, không merge
            grouped[sentence].append('')
    merged_sentences = []
    for prefix, suffixes in grouped.items():
        if all(not s for s in suffixes):  # chỉ có prefix, không có phần đuôi
            merged_sentences.append(prefix)
        else:
            merged = prefix + ' ' + ', '.join(s for s in suffixes if s)
            merged_sentences.append(merged)
    return '. '.join(merged_sentences) + '.'

In [ ]:
# Đường dẫn đến ONNX model và tokenizer
onnx_model_path = "/content/drive/MyDrive/ChatBot/model/bartpho/bartpho_qa.onnx"
tokenizer_path = "/content/drive/MyDrive/ChatBot/model/bartpho"

# Văn bản đầu vào chứa nhiều câu hỏi
input_text = "Triệu chứng bệnh Tiểu đường là gì?"

questions = [
    re.sub(r'[?.!]+$', '', q).strip()
    for q in re.split(r'[.?!]\s*', input_text)
    if q.strip()
]


# Xử lý từng câu hỏi
for question in questions:
    print(f"\nCâu hỏi: {question}?")
    contexts = get_top_topic_contexts(question)
    answer = test_generate_answer(question, contexts, onnx_model_path, tokenizer_path)

    # Xử lý gộp các câu giống nhau
    merged_answer = merge_similar_sentences(answer)

    print(f"✅ Câu trả lời: {merged_answer}")



Câu hỏi: Triệu chứng bệnh Tiểu đường là gì?
['Triệu chứng bệnh Tiểu đường thai kỳ']
['Triệu chứng bệnh Tiểu đường tuýp 1']
✅ Câu trả lời: Triệu chứng bệnh tiểu đường thai kỳ là hầu như không có triệu chứng đặc biệt, cần xét nghiệm để chẩn đoán, khát nước, tiểu nhiều, mệt mỏi, đường huyết tăng cao sau ăn. Triệu chứng bệnh tiểu đường tuýp 1 là hôn mê nhiễm toan ceton: yếu, mệt mỏi, khát nước, khô da, chuột rút, mạch nhanh, tụt huyết áp, rối loạn ý thức, buồn nôn, thở nhanh, hơi thở mùi táo thối, nhìn mờ, đau ngực không điển hình, tê bì dị cảm ở bàn chân, loét, nhiễm trùng bàn chân, đầy bụng, chậm tiêu, nuốt khó.


## ĐÁNH GIÁ TRUY XUẤT

In [ ]:
# load data test vinmec


In [ ]:
from rouge_score import rouge_scorer
from nltk.translate.bleu_score import sentence_bleu, SmoothingFunction

def evaluate_context_retrieval(retrieved_contexts, ground_truth_contexts):
    assert len(retrieved_contexts) == len(ground_truth_contexts), "Số lượng context không khớp"

    rouge = rouge_scorer.RougeScorer(['rouge1', 'rouge2', 'rougeL'], use_stemmer=True)
    smooth_fn = SmoothingFunction().method4

    rouge_scores = {'rouge1': 0, 'rouge2': 0, 'rougeL': 0}
    bleu_scores = []

    for retrieved, ground_truth in zip(retrieved_contexts, ground_truth_contexts):
        # Tính ROUGE
        scores = rouge.score(ground_truth, retrieved)
        rouge_scores['rouge1'] += scores['rouge1'].fmeasure
        rouge_scores['rouge2'] += scores['rouge2'].fmeasure
        rouge_scores['rougeL'] += scores['rougeL'].fmeasure

        # Tính BLEU
        reference = [ground_truth.split()]
        candidate = retrieved.split()
        bleu = sentence_bleu(reference, candidate, smoothing_function=smooth_fn)
        bleu_scores.append(bleu)

    num_examples = len(retrieved_contexts)
    avg_rouge_scores = {key: value / num_examples for key, value in rouge_scores.items()}
    avg_bleu_score = sum(bleu_scores) / num_examples

    return avg_rouge_scores, avg_bleu_score


In [ ]:
retrieved_contexts = val_df['retrieved_context'].tolist()   # context do RAG truy xuất
ground_truth_contexts = val_df['ground_truth_context'].tolist()  # context chuẩn từ dữ liệu

avg_rouge, avg_bleu = evaluate_context_retrieval(retrieved_contexts, ground_truth_contexts)

print("Average ROUGE:", avg_rouge)
print("Average BLEU:", avg_bleu)
